In [ ]:
pip install pandas scikit-learn nltk rank_bm25 jsonlines

In [ ]:
import jsonlines
import pandas as pd
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Load JSONL Data
def load_data(file_path):
    data = []
    try:
        with jsonlines.open(file_path) as reader:
            for obj in reader:
                data.append(obj)
    except jsonlines.InvalidLineError as e:
        print(f"Error parsing JSONL file: {e}")
        # Print the problematic line for debugging:
        print(f"Problematic line: {e.line}")
        # If you know the error, you can add logic to fix it here.
        # For example, if it's a missing comma, you could try to insert it.
        return pd.DataFrame([])  # Return empty DataFrame in case of error
    return pd.DataFrame(data)

# Load Train & Test Data
train_df = load_data("train.json")
test_df = load_data("test.json")

# Check data structure
print(train_df.head())

Error parsing JSONL file: line contains invalid json: unexpected end of data: line 2 column 1 (char 2) (line 1)
Problematic line: [
Error parsing JSONL file: line contains invalid json: unexpected end of data: line 2 column 1 (char 2) (line 1)
Problematic line: [
Empty DataFrame
Columns: []
Index: []


In [ ]:
pip install pandas


ERROR: Operation cancelled by user


In [ ]:
import pandas as pd

# Load JSON file into DataFrame
df = pd.read_json("train.json")

# Display DataFrame
print(df.head())


                                               claim  required_reannotation  \
0  Hunter Biden had no experience in Ukraine or i...                  False   
1  Donald Trump delivered the largest tax cuts in...                  False   
2  In Nigeria … in terms of revenue share, 20% go...                  False   
3  Biden has pledged to stop border wall construc...                  False   
4  After the police shooting of Jacob Blake, Gov....                  False   

       label                                      justification claim_date  \
0  Supported                       No former experience stated.  25-8-2020   
1    Refuted  Three tax bills have been larger than that of ...  25-8-2020   
2  Supported  The answer and source shows that the claim is ...  25-8-2020   
3  Supported  This claim should have been split into two par...  25-8-2020   
4    Refuted  Governor Evers did call for peace in a video s...  25-8-2020   

                   speaker                              

In [ ]:
import pandas as pd

# Load JSON file into DataFrame
df = pd.read_json("test.json")

# Display DataFrame
print(df.head())

                                               claim  claim_id  claim_date  \
0  Every Democrat on the Energy and Commerce Comm...         0  22-12-2021   
1  Meat packing workers have suffered more COVID-...         1   3-12-2021   
2  South African hospital found that traces of th...         2   3-12-2021   
3  Pfizer is “managed by Black Rock (sic) finance...         3   2-12-2021   
4  The Chinese biological laboratory in Wuhan is ...         4   2-12-2021   

           speaker                                 original_claim_url  \
0   LAUREN GIELLA   https://web.archive.org/web/20210623232457/htt...   
1  Bruce Blackburn                                               None   
2             None                                               None   
3      Susan Mundy    https://archive.ph/TkAYD#selection-346.0-767.42   
4      Susan Mundy    https://archive.ph/TkAYD#selection-346.0-767.42   

  reporting_source location_ISO_code  
0          twitter                US  
1         Face

In [ ]:
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the missing punkt_tab data

STOPWORDS = set(stopwords.words('english'))

# Text Cleaning Function
def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenization
    tokens = [word for word in tokens if word not in STOPWORDS]  # Remove stopwords
    return " ".join(tokens)

# Apply Preprocessing
df["clean_claim"] = df["claim"].apply(preprocess_text)
print(df.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


                                               claim  claim_id  claim_date  \
0  Every Democrat on the Energy and Commerce Comm...         0  22-12-2021   
1  Meat packing workers have suffered more COVID-...         1   3-12-2021   
2  South African hospital found that traces of th...         2   3-12-2021   
3  Pfizer is “managed by Black Rock (sic) finance...         3   2-12-2021   
4  The Chinese biological laboratory in Wuhan is ...         4   2-12-2021   

           speaker                                 original_claim_url  \
0   LAUREN GIELLA   https://web.archive.org/web/20210623232457/htt...   
1  Bruce Blackburn                                               None   
2             None                                               None   
3      Susan Mundy    https://archive.ph/TkAYD#selection-346.0-767.42   
4      Susan Mundy    https://archive.ph/TkAYD#selection-346.0-767.42   

  reporting_source location_ISO_code  \
0          twitter                US   
1         Fa

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Transform the claim text into numerical vectors
X = vectorizer.fit_transform(df["clean_claim"])

# Convert labels into numerical values
label_map = {"Supported": 0, "Refuted": 1, "Not Enough Evidence": 2, "Conflicting Evidence/Cherry-picking": 3}
df["label_encoded"] = df["label"].map(label_map)

# Display shape of the dataset
print(f"Feature Matrix Shape: {X.shape}")


KeyError: 'label'

In [ ]:
print(df.columns)


Index(['claim', 'claim_id', 'claim_date', 'speaker', 'original_claim_url',
       'reporting_source', 'location_ISO_code', 'clean_claim'],
      dtype='object')


In [ ]:
import json

# Load the JSON file and print an example entry
with open("train.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Print the first entry
print(data[0].keys())  # This will show all available keys in the dataset


dict_keys(['claim', 'required_reannotation', 'label', 'justification', 'claim_date', 'speaker', 'original_claim_url', 'fact_checking_article', 'reporting_source', 'location_ISO_code', 'claim_types', 'fact_checking_strategies', 'questions', 'cached_original_claim_url'])


In [ ]:
import pandas as pd
import json

# Load JSON file
with open("train.json", "r", encoding="utf-8") as f:
    data = json.load(f)  # Load full JSON list

# Convert JSON list to DataFrame
df = pd.DataFrame(data)

# Check if "label" is now included
print(df.columns)


Index(['claim', 'required_reannotation', 'label', 'justification',
       'claim_date', 'speaker', 'original_claim_url', 'fact_checking_article',
       'reporting_source', 'location_ISO_code', 'claim_types',
       'fact_checking_strategies', 'questions', 'cached_original_claim_url'],
      dtype='object')


In [ ]:
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the missing punkt_tab data

STOPWORDS = set(stopwords.words('english'))

# Text Cleaning Function
def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenization
    tokens = [word for word in tokens if word not in STOPWORDS]  # Remove stopwords
    return " ".join(tokens)

# Apply Preprocessing
df["clean_claim"] = df["claim"].apply(preprocess_text)


# Convert labels into numerical values, handling unknown labels
label_map = {"Supported": 0, "Refuted": 1, "Not Enough Evidence": 2, "Conflicting Evidence/Cherry-picking": 3}
# If a label is not found in label_map, it will be assigned -1 or any other suitable value
df["label_encoded"] = df["label"].map(label_map).fillna(-1).astype(int)
#fillna replaces NaN with -1. astype(int) converts to integers.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Transform the claim text into numerical vectors
X = vectorizer.fit_transform(df["clean_claim"])

# Convert labels into numerical values
label_map = {"Supported": 0, "Refuted": 1, "Not Enough Evidence": 2, "Conflicting Evidence/Cherry-picking": 3}
df["label_encoded"] = df["label"].map(label_map)

# Display shape of the dataset
print(f"Feature Matrix Shape: {X.shape}")


Feature Matrix Shape: (3068, 5000)


In [ ]:
print(df["label"].isna().sum())  # Count NaN values in label column


0


In [ ]:
df = df.dropna(subset=["label"])


In [ ]:
df["label"].fillna("Not Enough Evidence", inplace=True)


<ipython-input-25-3c7f571bbd70>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["label"].fillna("Not Enough Evidence", inplace=True)


In [ ]:
print("NaN values in label column:", df["label"].isna().sum())  # Check NaNs in the original label column
print("NaN values in label_encoded column:", df["label_encoded"].isna().sum())  # Check NaNs in the encoded label column


NaN values in label column: 0
NaN values in label_encoded column: 195


In [ ]:
# Find unique labels in the dataset
print("Unique labels in dataset:", df["label"].unique())

# Find labels that were not mapped correctly
unmapped_labels = df[df["label_encoded"].isna()]["label"].unique()
print("Labels that caused NaN in encoding:", unmapped_labels)


Unique labels in dataset: ['Supported' 'Refuted' 'Conflicting Evidence/Cherrypicking'
 'Not Enough Evidence']
Labels that caused NaN in encoding: ['Conflicting Evidence/Cherrypicking']


In [ ]:
label_map = {
    "Supported": 0,
    "Refuted": 1,
    "Not Enough Evidence": 2,
    "Conflicting Evidence/Cherry-picking": 3
}

# If there are variations in spelling, add them to the map
df["label_encoded"] = df["label"].map(label_map)

# Fill remaining NaN labels with "Not Enough Evidence" (default class)
df["label_encoded"].fillna(2, inplace=True)  # Assign 2 = "Not Enough Evidence"

# Convert to integer
df["label_encoded"] = df["label_encoded"].astype(int)


<ipython-input-29-740c3218c58f>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["label_encoded"].fillna(2, inplace=True)  # Assign 2 = "Not Enough Evidence"


In [ ]:
print("NaN values in label_encoded after fixing:", df["label_encoded"].isna().sum())  # Should be 0


NaN values in label_encoded after fixing: 0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, df["label_encoded"], test_size=0.2, random_state=42)

# Train Model
clf = LogisticRegression(max_iter=500)
clf.fit(X_train, y_train)

# Make Predictions
y_pred = clf.predict(X_test)

# Model Evaluation
print("Model Evaluation:")
print(classification_report(y_test, y_pred))

Model Evaluation:
              precision    recall  f1-score   support

           0       0.71      0.40      0.51       167
           1       0.66      0.95      0.78       359
           2       0.40      0.02      0.04        88

    accuracy                           0.67       614
   macro avg       0.59      0.46      0.44       614
weighted avg       0.64      0.67      0.60       614



In [ ]:
from rank_bm25 import BM25Okapi

# Tokenize evidence text
df["tokenized_evidence"] = df["evidence"].apply(lambda x: nltk.word_tokenize(str(x).lower()))

# Initialize BM25 Model
bm25 = BM25Okapi(df["tokenized_evidence"])

# Function to Retrieve Best Evidence for a Claim
def retrieve_best_evidence(claim):
    tokenized_claim = nltk.word_tokenize(claim.lower())
    scores = bm25.get_scores(tokenized_claim)
    best_evidence_idx = scores.argmax()
    return df.iloc[best_evidence_idx]["evidence"]fg,ktdiu8rex7w4z

# Apply evidence retrieval to test set
df["retrieved_evidence"] = df["clean_claim"].apply(retrieve_best_evidence)

print("Evidence Retrieval Completed!")


KeyError: 'evidence'

ev2r score


In [ ]:
# Predict labels using trained Logistic Regression model
y_pred = clf.predict(X_test)

# Convert numerical predictions back to label names
reverse_label_map = {0: "Supported", 1: "Refuted", 2: "Not Enough Evidence", 3: "Conflicting Evidence/Cherry-picking"}
df_test["predicted_label"] = [reverse_label_map[p] for p in y_pred]

# Check structure
print(df_test[["claim", "predicted_label"]].head())


NameError: name 'df_test' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, df["label_encoded"], test_size=0.2, random_state=42)


In [ ]:
df_test = df.iloc[y_test.index]  # Use y_test.index instead of X_test.index


In [ ]:
# Convert numerical predictions back to text labels
reverse_label_map = {0: "Supported", 1: "Refuted", 2: "Not Enough Evidence", 3: "Conflicting Evidence/Cherry-picking"}

# Assign predicted labels
df_test["predicted_label"] = [reverse_label_map[p] for p in y_pred]

# Check structure
print(df_test[["claim", "predicted_label"]].head())


                                                  claim predicted_label
2650  South Africa has launched a Youth Employment S...       Supported
1739  A Muslim man in India was killed by a mob afte...         Refuted
2605  The number of people on health insurance in Ke...       Supported
1455  The ExIm Bank provided taxpayer-backed handout...         Refuted
2163  £22 billion was taken out of welfare spending ...         Refuted


<ipython-input-39-47f92c07fb20>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["predicted_label"] = [reverse_label_map[p] for p in y_pred]


In [ ]:
# Verify that retrieved and reference evidence exist
print(df_test[["retrieved_evidence", "reference_evidence"]].head())


KeyError: "None of [Index(['retrieved_evidence', 'reference_evidence'], dtype='object')] are in the [columns]"